In [1]:
#https://www.kaggle.com/code/rtatman/beginner-s-intro-to-rnn-s-in-r/notebook

# https://www.youtube.com/watch?v=Ttm28BhmAcs 
# read in the packages we'll use
library(keras) # for deep learning
library(tidyverse) # general utility functions
library(pROC)

In [2]:
setwd("/kaggle/input/basetables")
load("basetable_test.RData")
load("basetable_train.RData")

In [ ]:
normalize <- function(x, min = min, max = max, na.rm = TRUE) {
    return((x- min) /(max-min))
}

basetable_train$volume <- normalize(basetable_train$volume, min(basetable_train$volume), max(basetable_train$volume))
basetable_train$ema <- normalize(basetable_train$ema, min(basetable_train$ema), max(basetable_train$ema))
basetable_train$rsi <- normalize(basetable_train$rsi, min(basetable_train$rsi), max(basetable_train$rsi))

basetable_test$volume <- normalize(basetable_test$volume, min(basetable_train$volume), max(basetable_train$volume))
basetable_test$ema <- normalize(basetable_test$ema, min(basetable_train$ema), max(basetable_train$ema))
basetable_test$rsi <- normalize(basetable_test$rsi, min(basetable_train$rsi), max(basetable_train$rsi))

In [3]:
basetable_train$Target <- ifelse(basetable_train$Target < 0, 0, 1)
basetable_test$Target <- ifelse(basetable_test$Target < 0, 0, 1)

In [4]:
basetable_train <- subset(basetable_train, select = -c(volume, ema, M, ROC, rsi))
basetable_test <- subset(basetable_test, select = -c(volume, ema, M, ROC, rsi))

In [ ]:
download.csv <- function(dataframe) {
    # function that takes in a dataframe
    # and creates a text link download it
    # without saving in R or committing the kernel
    # (will only work for files cca < 2 MB)
    
    # create CSV vector from the dataframe
    csv.vector <- capture.output(write.csv(dataframe))
    
    # create CSV string before converting to base64
    csv.string <- paste(csv.vector, collapse="\n")
    
    # create base64 encoded CSV
    csv.base64 <- as.character(RCurl::base64(csv.string))
    
    # create HTML link
    csv.html <- paste("<a download=\"dataframe.csv\" href=\"data:text/csv;base64,", csv.base64, "\" target=\"_blank\">Download CSV</a>", sep="")
    
    # parse output at HTML
    IRdisplay::display_html(csv.html)
}

In [ ]:
download.csv(basetable_train)

In [ ]:
download.csv(basetable_test)

In [5]:
set.seed(16)

val_dates <- sample(unique(basetable_train$date), length(unique(basetable_train$date))*0.2)
dates <- unique(basetable_train$date)
train_dates <- dates[!(dates %in% val_dates)]
basetable_val <- basetable_train[basetable_train$date %in% val_dates,]
basetable_train_all <- basetable_train
basetable_train <- basetable_train[basetable_train$date %in% train_dates,]

In [6]:
train_X <- as.matrix(subset(basetable_train, select = -c(date, Target)))
train_all_X <- as.matrix(subset(basetable_train_all, select = -c(date, Target)))
val_X <- as.matrix(subset(basetable_val, select = -c(date, Target)))
test_X <- as.matrix(subset(basetable_test, select = -c(date, Target)))

train_y <- as.matrix(basetable_train$Target)
train_all_y <- as.matrix(basetable_train_all$Target)
val_y <- as.matrix(basetable_val$Target)
test_y <- as.matrix(basetable_test$Target)

In [140]:
model <- keras_model_sequential() %>%
    layer_dense(units = 8, input_shape = c(6), kernel_constraint = constraint_maxnorm(1)) %>%
    layer_activation_leaky_relu() %>%
    layer_dense(units = 4, kernel_constraint = constraint_maxnorm(1)) %>%
    layer_activation_leaky_relu() %>%
    layer_dense(units = 1, activation = "sigmoid")
    
opt <- optimizer_adam(lr = 0.001)
    
model %>% compile(
    optimizer = opt,
    loss = "binary_crossentropy",
    metrics = c("accuracy"))

history <- model %>% fit(
    train_X,
    train_y,
    epochs = 30,
    batch_size = 16,
    validation_data = list(val_X, val_y),
    verbose=1
)

model %>% summary()

setwd('/kaggle/working/')
plot(history)

In [141]:
model <- keras_model_sequential() %>%
    layer_dense(units = 8, input_shape = c(6), kernel_constraint = constraint_maxnorm(1)) %>%
    layer_activation_leaky_relu() %>%
    layer_dense(units = 4, kernel_constraint = constraint_maxnorm(1)) %>%
    layer_activation_leaky_relu() %>%
    layer_dense(units = 1, activation = "sigmoid")

model %>% compile(
    optimizer = opt,
    loss = "binary_crossentropy",
    metrics = c("accuracy")
)

history <- model %>% fit(train_all_X, train_all_y, epochs = 15, batch_size = 20)
test_y_hat <- model %>% predict(test_X, type = "response")

plot(history)

In [142]:
test_y_hat

In [143]:
auc(test_y, test_y_hat)
plot.roc(test_y,test_y_hat, print.auc = TRUE, col = "red", ylim = c(0,1))